In [ ]:
source(paste0(dirname(getwd()),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))

In [ ]:
annotation <- fread(paste0(dirname(getwd()), "/util/annotation/pathways.csv"))

### 0 - Read and process collected driver data
- Just including all drivers output from Linx here

In [ ]:
annotation <- fread(paste0(dirname(getwd()), "/util/annotation/pathways.csv"))

In [ ]:
drivers <- 
fread( paste0(TMP_DIR, "drivers.csv")) %>% 
  lj(annotation, by = "gene") %>% 
  mu(likelihood = ifelse(category == "TSG" & likelihoodMethod == "DISRUPTION", 1, driverLikelihood)) %>% 
  fi(likelihood > .8)  

### 1 - Prep drivers 

In [ ]:
drivers_ready <- 
drivers %>% 
  transmute(sampleId, gene = paste0("driver_", gene)) %>% 
  unique() %>% 
  mutate(driver = 1) %>% 
  spread(gene, driver)

In [ ]:
drivers_ready[is.na(drivers_ready)] <- 0

### 2 - Compute Pathway counts 

In [ ]:
pathways_ready <- 
drivers %>% 
 gb(sampleId, pathway) %>% 
 su(tot = n()) %>% 
 ug() %>% 
 tm(sampleId, pathway = paste0("drivers_pathway_", pathway), tot) %>% 
 unique() %>% 
 spread(pathway, tot)

In [ ]:
pathways_ready[is.na(pathways_ready)] <- 0

### 3 - Total Drivers and Total Pathways

In [ ]:
total_drivers <- 
drivers %>% 
  gb(sampleId) %>% 
  su(drivers_total = n()) 

In [ ]:
total_pathways <- 
drivers %>% 
 gb(sampleId) %>% 
 su(drivers_pathway_total = n_distinct(pathway))

### 4 - Send the drivers file

In [ ]:
together <- 
drivers_ready %>% 
 ij(pathways_ready, by = "sampleId") %>% 
 ij(total_drivers, by = "sampleId") %>% 
 ij(total_pathways, by = "sampleId")

In [ ]:
fwrite(together, paste0(READY_DIR, "drivers_ready.csv"))